# Paths

In [0]:
PATH_WORKING_SPACE = "/content/gdrive/My Drive/Colab Notebooks/"

In [2]:
PATH_SEQUENCE = PATH_WORKING_SPACE + "sequence.ipynb"
print(PATH_SEQUENCE)

/content/gdrive/My Drive/Colab Notebooks/sequence.ipynb


# Mount drive & run additional scripts

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
def load(path, input_shape):

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        channel_shift_range=0.3,
        brightness_range= [0.7, 1.3],
        fill_mode='nearest')
    BATCH_SIZE=32
    print('batchsize', BATCH_SIZE)
    train_it = datagen.flow_from_directory('{0}train'.format(path),
                                           class_mode='categorical',
                                           target_size=input_shape,
                                           batch_size=BATCH_SIZE,
                                           color_mode='rgb',
                                           classes=['hats', 'nothats'])
    val_it = datagen.flow_from_directory('{0}val'.format(path),
                                          class_mode='categorical',
                                          target_size=input_shape,
                                          batch_size=BATCH_SIZE,
                                          color_mode='rgb',
                                          classes=['hats', 'nothats'])

    return train_it, val_it

# Enable GPU

In [6]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Training Binary Classifier for hats

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras.losses
import keras.optimizers
import numpy as np

from sklearn.model_selection import train_test_split
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet

def hatmodel(input_shape, num_classes, verbose=False):
    model = Sequential()
    model.add(keras.applications.mobilenet.MobileNet(weights='imagenet',include_top=False))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(64, activation='relu')) # 64
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu')) # 128
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu')) #256
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.layers[0].trainable=False

    model.summary()
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    if verbose:
        model.summary()

    return model


if __name__ == '__main__':
    dims = (224, 224, 3)
    onehotclasses = 2
    epochs = 10
    mdl = hatmodel(dims, onehotclasses)?authuser

    train_gen, val_gen = load('gdrive/My Drive/mimhats/dataset/', (224,224))


    mdl.fit_generator(
        generator=train_gen,
        validation_data=val_gen,
        epochs=epochs,
        verbose=True
    )


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, None, None, 1024)  3228864   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)             

# Save weight & architecture

In [0]:
mdl.save('mimuw_hats_classifier.h5')